#### ETL relaizado sobre el archivo `user.parquet`

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
parquet_file_path = "c:/Users/PC/Documents/RawDataPF_Henry/Yelp/user.parquet"
df_user_Yelp = pd.read_parquet(parquet_file_path)

In [3]:
df_user_Yelp.head()

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [4]:
df_user_Yelp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 353.4+ MB


In [5]:
# Eliminar las columnas "yelping_since" y "friends" del DataFrame df_user_Yelp
df_user_Yelp = df_user_Yelp.drop(columns=['yelping_since', 'friends'])

In [57]:
# df_user_Yelp.to_csv("c:/Users/PC/Documents/RawDataPF_Henry/Yelp_users.csv")

A continuación se realizan algunas transformaciones tomando la información que hay en común con el archivo 'business_final_csv' que contiene sólo la información de los 3 estados de interés (FL, NV y CA).<br>
Primero uno 'Yelp_reviews' con 'business_final' para reducir el número de reviews, dejando sólo los que corresponden a los business de los estados mencionados

In [6]:
df_review_Yelp = pd.read_csv('c:/Users/PC/Documents/RawDataPF_Henry/Yelp_reviews.csv')

In [7]:
df_business = pd.read_csv("c:/Users/PC/Documents/RawDataPF_Henry/business_final.csv")

In [11]:
# Antes del merge, me aseguro de que las columnas 'business_id' estén en un formato consistente
df_review_Yelp['business_id'] = df_review_Yelp['business_id'].str.strip()
df_business['business_id'] = df_business['business_id'].str.strip()

# Realizar el merge basado en la columna "business_id"
merged_df = df_review_Yelp.merge(df_business[['business_id']], on='business_id', how='inner')

# Seleccionar las columnas necesarias en el DataFrame resultante
columns_to_keep = ['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date']
df_review_Yelp_final = merged_df[columns_to_keep]

# Ahora, df_review_Yelp_final contendrá solo las filas de df_review_Yelp que tienen "business_id" presente en df_business,
# y solo las columnas seleccionadas.

Lo mismo se realiza con 'Yelp_users' para que contenga sólo la información de los 'user_id' presentes en 'review_Yelp_final'

In [12]:
# Realizar la fusión (merge) basada en la columna 'user_id'
df_user_Yelp_filt = df_user_Yelp[df_user_Yelp['user_id'].isin(df_review_Yelp_final['user_id'])]

# Ahora, df_user_Yelp_filt contendrá solo las filas de df_user_Yelp con los 'user_id' presentes en df_review_Yelp_final.


In [80]:
df_user_Yelp_filt.to_csv("c:/Users/PC/Documents/RawDataPF_Henry/Yelp_users_final.csv")

In [86]:
df_user_Yelp_filt.info()

<class 'pandas.core.frame.DataFrame'>
Index: 932993 entries, 0 to 2105596
Data columns (total 20 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   user_id             932993 non-null  object 
 1   name                932993 non-null  object 
 2   review_count        932993 non-null  int64  
 3   useful              932993 non-null  int64  
 4   funny               932993 non-null  int64  
 5   cool                932993 non-null  int64  
 6   elite               932993 non-null  object 
 7   fans                932993 non-null  int64  
 8   average_stars       932993 non-null  float64
 9   compliment_hot      932993 non-null  int64  
 10  compliment_more     932993 non-null  int64  
 11  compliment_profile  932993 non-null  int64  
 12  compliment_cute     932993 non-null  int64  
 13  compliment_list     932993 non-null  int64  
 14  compliment_note     932993 non-null  int64  
 15  compliment_plain    932993 non-null  i

In [ ]:
# Scrip enviar archivos al contenedor de Azure desde python
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

def upload_to_azure_blob(account_name, account_key, container_name, local_file_path, blob_name):
    # Crear el cliente del servicio de Blob
    blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net/", credential=account_key)

    # Obtener una referencia al contenedor
    container_client = blob_service_client.get_container_client(container_name)

    # Cargar el archivo local en Azure Blob Storage
    with open(local_file_path, "rb") as data:
        blob_client = container_client.get_blob_client(blob_name)
        blob_client.upload_blob(data)

# Variables de configuración
account_name = 'proyectofinalgrupo05'
account_key = 'vrh6zC8lshFY666G21tX7B5jFBsTKcEpv90E6ttyPVnbckIheKpL3zSnNOse9YrTSoHmtULIcSEu+ASt0qK2EA=='
container_name = 'datasetscontainer'
local_file_path = r"c:\Users\PC\Documents\RawDataPF_Henry\Yelp_users_final.csv"
blob_name = 'Yelp_users.csv'

# Llamar a la función para cargar el archivo en Azure Blob Storage
upload_to_azure_blob(account_name, account_key, container_name, local_file_path, blob_name)